In [1]:
import sys
import os
import gzip
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
def get_median(a):
    a.sort()
    centerIndex = len(a) // 2 
    if len(a)% 2 == 1:
        return a[centerIndex ]
    return ((a[centerIndex - 1] + a[centerIndex]) / 2)
from matplotlib_venn import venn3, venn3_circles
import matplotlib.pyplot as plt 
from matplotlib_venn import venn2, venn2_circles
from tqdm import tqdm

/opt/Yonsei/python/3.8.1/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.2' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


### Germline concordance pre-process

In [2]:
sample_name_list = [line.rstrip('\n') for line in open('/data/project/Meningioma/script/sample_name.txt', 'r')]
sample_name_list = ["190426", "220930", "221026", "221102", "221202", "230127", "230303", "230323_2" ]
HC_DIR="/data/project/Meningioma/06.hc/01.call"

input_file = []
for Sample_ID in sample_name_list:
    for TISSUE in ["Blood", "Tumor", "Dura"]: # 각각에 대해서 다 돌려보고 germline concordance도 확인해보자
        input_dir = HC_DIR + "/" + Sample_ID + "/" + TISSUE
        input_file.append (input_dir + "/" + Sample_ID + "_" + TISSUE + ".vcf")

input_file.sort()


whole_dic = {}
name_list = []
for file in tqdm(input_file):
    name = file.split("/")[-1].split(".")[0]
    name_list.append(name)
    whole_dic[name] = set()
    
    f = open( file, 'r' )
    for fline in f:
        if fline.startswith('#'):
            continue
        ftab = fline.rstrip().split('\t')
        ID = ftab[0] + '_' + ftab[1]    #"CHR.POS"
        whole_dic[name].add(ID)
    f.close()

print (name_list)

  0%|          | 0/24 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: '/data/project/Meningioma/06.hc/01.call/190426/Blood/190426_Blood.vcf'

### Germline concordance running

In [6]:
first_list = list(name_list)
second_list = list(name_list)

output_dir = "/data/project/Meningioma/script/04.Shared_calls"
os.makedirs(output_dir, exist_ok = True)

fo = open(output_dir + "/germline_concordance.txt",'w')
fo.write('concor' + '\t')
for first in first_list:
    if first == first_list[-1]:
        fo.write(first + '\n')
    else:
        fo.write(first + '\t')

for first in first_list:
    fo.write(first + '\t')
    for second in second_list:
        if second == second_list[-1]:
            if first == second:
                fo.write('100' + '\n')
                print(first, second, '100')
            else:
                concordance = round(len(whole_dic[first] & whole_dic[second])/len(whole_dic[first] | whole_dic[second]) * 100, 2)
                fo.write(str(concordance) + '\n')
                print(first, second, str(concordance))

        else:
            if first == second:
                fo.write('100' + '\t')
                print(first, second, '100')
            else:
                concordance = round(len(whole_dic[first] & whole_dic[second])/len(whole_dic[first] | whole_dic[second]) * 100, 2)
                fo.write(str(concordance) + '\t')
fo.close()

220930_Blood 220930_Blood 100
220930_Blood 230419_Tumor 51.72
220930_Dura 220930_Dura 100
220930_Dura 230419_Tumor 51.85
220930_Tumor 220930_Tumor 100
220930_Tumor 230419_Tumor 51.52
221026_Blood 221026_Blood 100
221026_Blood 230419_Tumor 51.77
221026_Dura 221026_Dura 100
221026_Dura 230419_Tumor 51.94
221026_Tumor 221026_Tumor 100
221026_Tumor 230419_Tumor 51.71
221102_Blood 221102_Blood 100
221102_Blood 230419_Tumor 52.46
221102_Dura 221102_Dura 100
221102_Dura 230419_Tumor 52.5
221102_Tumor 221102_Tumor 100
221102_Tumor 230419_Tumor 52.44
230127_Blood 230127_Blood 100
230127_Blood 230419_Tumor 52.34
230127_Dura 230127_Dura 100
230127_Dura 230419_Tumor 52.37
230127_Tumor 230127_Tumor 100
230127_Tumor 230419_Tumor 52.22
230323_Blood 230323_Blood 100
230323_Blood 230419_Tumor 52.35
230323_Dura 230323_Dura 100
230323_Dura 230419_Tumor 52.32
230323_Tumor 230323_Tumor 100
230323_Tumor 230419_Tumor 52.43
230419_Blood 230419_Blood 100
230419_Blood 230419_Tumor 97.53
230419_Dura 230419_Dura 

In [7]:
cor_df = pd.read_csv (output_dir + "/germline_concordance.txt", sep ="\t")
cor_df.style.background_gradient(cmap ='coolwarm').set_precision(1)

# corr = round ( result.NUM_CLONE_record.iloc[ : , 1:].astype ( int).corr(),  2)
# corr.style.background_gradient(cmap='coolwarm', axis=None).set_precision(2)

<ipython-input-7-0b23d4540ed4>:2: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  cor_df.style.background_gradient(cmap ='coolwarm').set_precision(1)


,concor,220930_Blood,220930_Dura,220930_Tumor,221026_Blood,221026_Dura,221026_Tumor,221102_Blood,221102_Dura,221102_Tumor,230127_Blood,230127_Dura,230127_Tumor,230323_Blood,230323_Dura,230323_Tumor,230419_Blood,230419_Dura,230419_Tumor
0,220930_Blood,100.0,98.0,97.2,52.4,52.5,52.3,52.7,52.7,52.6,52.2,52.2,52.0,52.4,52.5,52.2,52.0,52.0,51.7
1,220930_Dura,98.0,100.0,97.3,52.4,52.7,52.3,52.7,52.8,52.6,52.2,52.3,52.1,52.5,52.6,52.3,52.1,52.1,51.9
2,220930_Tumor,97.2,97.3,100.0,52.2,52.2,52.1,52.4,52.4,52.4,51.9,51.9,51.7,52.1,52.2,52.0,51.7,51.7,51.5
3,221026_Blood,52.4,52.4,52.2,100.0,97.5,97.1,52.7,52.7,52.8,52.2,52.3,52.2,52.0,52.1,51.9,52.0,52.0,51.8
4,221026_Dura,52.5,52.7,52.2,97.5,100.0,97.2,52.9,53.0,52.8,52.5,52.6,52.3,52.2,52.4,52.1,52.1,52.3,51.9
5,221026_Tumor,52.3,52.3,52.1,97.1,97.2,100.0,52.6,52.6,52.6,52.1,52.2,52.0,51.9,52.0,51.8,51.8,51.9,51.7
6,221102_Blood,52.7,52.7,52.4,52.7,52.9,52.6,100.0,98.0,97.8,52.8,52.9,52.6,52.4,52.6,52.2,52.7,52.7,52.5
7,221102_Dura,52.7,52.8,52.4,52.7,53.0,52.6,98.0,100.0,97.7,52.9,53.0,52.6,52.5,52.6,52.3,52.7,52.8,52.5
8,221102_Tumor,52.6,52.6,52.4,52.8,52.8,52.6,97.8,97.7,100.0,52.7,52.8,52.6,52.4,52.5,52.2,52.7,52.6,52.4
9,230127_Blood,52.2,52.2,51.9,52.2,52.5,52.1,52.8,52.9,52.7,100.0,98.1,97.6,51.7,51.8,51.6,52.5,52.6,52.3
